In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import itertools


compute_dataset = pd.read_csv("../data/logfile-v5.csv", delimiter=";")
compute_dataset.head()

storage_dataset = pd.read_csv("../data/logfile-storage-v4.csv", delimiter=";")
storage_dataset.head()

network_dataset = pd.read_csv("../data/logfile-network-v4.csv", delimiter=";")
network_dataset.head()
labels = np.unique(compute_dataset['datacenter_name'])

/tmp/ipykernel_565498/1136695272.py:9: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  compute_dataset = pd.read_csv("../data/logfile-v5.csv", delimiter=";")


Investigating the effect of network traffic on updated workload and data placement.

### Simple Placement

Simple Resource Aware Workload Placement 

In [42]:

vms = []
hosts = []
for index, sample in compute_dataset[compute_dataset['time'] == 300.01].iterrows():
    hosts.append({
        "id": sample['host_id'],
        "datacenter_name": sample['datacenter_name'],
        "active": sample['active'],
        "power_model": sample['power_model'].split(","),
        "pes": int(sample['number_of_pes']),
        "mips": float(sample['mips']),
        "used_mips": (float(sample['mips']) - float(sample['available_mips'])),
        "ram": float(sample['ram']),
        "used_ram": (float(sample['ram']) - float(sample['available_ram'])),
        "bw": float(sample['bw']),
        "used_bw": (float(sample['bw']) - float(sample['available_bw'])),
    })
    if pd.notna(sample['vms']):
        for index, vm in enumerate(sample['vms'].split(":")):
            vm_allocated_resources = vm.split(",")
            if len(vm_allocated_resources) != 4:
                continue
            vms.append({
                "id": index,
                "pes": int(vm_allocated_resources[0]),
                "mips": float(vm_allocated_resources[1]),
                "ram": float(vm_allocated_resources[2]),
                "bw": float(vm_allocated_resources[3])
            })

vms_copy = vms.copy()
new_workloads_placement = []
for index_host, host in enumerate(hosts):
    used_mips = 0
    used_ram = 0
    used_bw = 0
    workloads_per_host = []
    
    for index_vm, vm in enumerate(vms_copy):
        if ((vm['mips'] + used_mips) <= host['mips']) and ((vm['ram'] + used_ram) <= host['ram']) and ((vm['bw'] + used_bw) <= host['bw']):
            used_mips += vm['mips']
            used_ram += vm['ram']
            used_bw += vm['bw']
            workloads_per_host.append(vm)
    
    new_workloads_placement.append(workloads_per_host)
    for index_vm, vm in enumerate(workloads_per_host):
        run = True
        index = 0
        while run:
            if vm['id'] == vms_copy[index]['id']:
                del vms_copy[index]
                run = False
            index += 1

#### Simple Capacity Aware Disk Placement

In [43]:

disks = []
files = []
for i, sample in storage_dataset.iterrows():
    disks.append({
        "id": i,
        "datacenter": sample['datacenter_name'],
        "storage_capacity": float(sample['storage_capacity']),
        "capacity_used": float(sample['capacity_used']),
        "active_power": float(sample['active_power']),
        "idle_power": float(sample['idle_power'])

    })
    if pd.notna(sample['files']):
        for j, file in enumerate(sample['files'].split(":")):
            file_data = file.split(",")
            if len(file_data) != 3:
                continue
            files.append({
                "id": j,
                "filename": file_data[0],
                "filesize": float(file_data[1]),
                "transaction_time": float(file_data[2])
            })

files_copy = files.copy()
new_storage_placement = []
removed_files_from_copy = 0

for index_disk, disk in enumerate(disks):
    used_storage = 0
    files_per_disk = []
    for file in files_copy:
        if file['filesize'] + used_storage <= disk['storage_capacity']:
            used_storage += file['filesize']
            files_per_disk.append(file)
    
    # Adding to new list of placement of data and removing from the copy 
    new_storage_placement.append(files_per_disk)
    for index, file in enumerate(files_per_disk):
        run = True
        index = 0
        while run:
            if file['id'] == files_copy[index]['id']:
                del files_copy[index]
                run = False
            index += 1


#### The Resulting Network Infrastructure 

In [44]:
switches = []
datacenters_switches = []

num_current_shutdown_ports = 0
num_current_shutdown_switches = 0

num_current_could_shutdown_ports = 0
num_current_could_shutdown_switches = 0
# vms
# hosts

# Powering down ports and switches connected to unused hosts, switches
for datacenter_name in labels:
    switches_per_datacenter = []
    for index_switches, sample in network_dataset[network_dataset['datacenter_name'] == datacenter_name].iterrows():
        ports = []
        if sample['role'] == "access switch":
            could_shutdown = True
            if pd.notna(sample['ports']):
                for index_port, port in enumerate(sample['ports'].split(":")):
                    if index_port == 0:
                        pass
                    elif port != "" and index_port != 0:
                        port_info = port.split(",") # ports: id, is_active, active_duration, switch_id (tx), host_id (rx), bandwidth, total_used_bandwidth    
                        for host in hosts:
                            if (datacenter_name == host['datacenter_name']) and (int(port_info[4]) == host['id']):
                                ports.append({
                                    "port_id": int(port_info[0]),
                                    "is_active": bool(port_info[1]),
                                    "active_duration": float(port_info[2]),
                                    "switch_id": int(port_info[3]),
                                    "host_id": int(port_info[4]),
                                    "host": host,
                                    "bandwidth": float(port_info[5]),
                                    "total_used_bandwidth": float(port_info[6])
                                })
                                if bool(port_info[1]) == False:
                                    num_current_shutdown_ports += 1
                                else:
                                    could_shutdown = False
                                if float(port_info[6]) == 0:
                                    num_current_could_shutdown_ports += 1
                                else:
                                    could_shutdown = False
            access_switch = {
                "switch_id": sample['id'],
                "datacenter_id": sample['datacenter_id'],
                "datacenter_name": sample['datacenter_name'],
                "role": sample['role'],
                "level": sample['level'],
                "bandwidth": sample['bandwidth'],
                "total_used_bandwidth": sample['total_used_bandwidth'],
                "traffic_history": sample['traffic_history'],
                "ports": ports,
                "is_active": sample['is_active'],
                "duration": sample['duration'],
                "power_per_port": sample['power_per_port'],
                "low_power": sample['low_power'],
                "high_power": sample['high_power'],
            }
            switches.append(access_switch)
            switches_per_datacenter.append(access_switch)
            
            if sample['is_active'] == False:
                num_current_shutdown_switches += 1
            if could_shutdown:
                num_current_could_shutdown_switches += 1
            
        elif sample['role'] == "distribution switch":
            could_shutdown = True
            if pd.notna(sample['ports']):
                for index_port, port in enumerate(sample['ports'].split(":")):
                    if index_port == 0:
                        pass
                    elif port != "" and index_port != 0:
                        port_info = port.split(",") # ports: id, is_active, active_duration, switch_id (tx), host_id (rx), bandwidth, total_used_bandwidth    
                        for switch in switches:
                            if (datacenter_name == switch['datacenter_name']) and (int(port_info[4]) == switch['switch_id']):
                                ports.append({
                                    "port_id": int(port_info[0]),
                                    "is_active": bool(port_info[1]),
                                    "active_duration": float(port_info[2]),
                                    "switch_current_level_id": int(port_info[3]),
                                    "switch_lower_level_id": int(port_info[4]),
                                    "switch": switch,
                                    "bandwidth": float(port_info[5]),
                                    "total_used_bandwidth":float (port_info[6])
                                })
                                if bool(port_info[1]) == False:
                                    num_current_shutdown_ports += 1
                                else:
                                    could_shutdown = False
                                if float(port_info[6]) == 0:
                                    num_current_could_shutdown_ports += 1
                                else:
                                    could_shutdown = False
            distribution_switch = {
                "switch_id": sample['id'],
                "datacenter_id": sample['datacenter_id'],
                "datacenter_name": sample['datacenter_name'],
                "role": sample['role'],
                "level": sample['level'],
                "bandwidth": sample['bandwidth'],
                "total_used_bandwidth": sample['total_used_bandwidth'],
                "traffic_history": sample['traffic_history'],
                "ports": ports,
                "is_active": sample['is_active'],
                "duration": sample['duration'],
                "power_per_port": sample['power_per_port'],
                "low_power": sample['low_power'],
                "high_power": sample['high_power'],
            }
            switches.append(distribution_switch)
            switches_per_datacenter.append(distribution_switch)
            
            if sample['is_active'] == False:
                num_current_shutdown_switches += 1
            if could_shutdown:
                num_current_could_shutdown_switches += 1
        elif sample['role'] == "core switch":
            could_shutdown = True
            if pd.notna(sample['ports']):
                for index_port, port in enumerate(sample['ports'].split(":")):
                    if port != "":
                        port_info = port.split(",") # ports: id, is_active, active_duration, switch_id (tx), host_id (rx), bandwidth, total_used_bandwidth    
                        for switch in switches:
                            if (datacenter_name == switch['datacenter_name']) and (int(port_info[4]) == switch['switch_id']):
                                ports.append({
                                    "port_id": int(port_info[0]),
                                    "is_active": bool(port_info[1]),
                                    "active_duration": float(port_info[2]),
                                    "switch_current_level_id": int(port_info[3]),
                                    "switch_lower_level_id": int(port_info[4]),
                                    "switch": switch,
                                    "bandwidth": float(port_info[5]),
                                    "total_used_bandwidth":float (port_info[6])
                                })
                                if bool(port_info[1]) == False:
                                    num_current_shutdown_ports += 1
                                else:
                                    could_shutdown = False
                                if float(port_info[6]) == 0:
                                    num_current_could_shutdown_ports += 1
                                else:
                                    could_shutdown = False
            core_switch = {
                "switch_id": sample['id'],
                "datacenter_id": sample['datacenter_id'],
                "datacenter_name": sample['datacenter_name'],
                "role": sample['role'],
                "level": sample['level'],
                "bandwidth": sample['bandwidth'],
                "total_used_bandwidth": sample['total_used_bandwidth'],
                "traffic_history": sample['traffic_history'],
                "ports": ports,
                "is_active": sample['is_active'],
                "duration": sample['duration'],
                "power_per_port": sample['power_per_port'],
                "low_power": sample['low_power'],
                "high_power": sample['high_power'],
            }
            switches.append(core_switch)
            switches_per_datacenter.append(core_switch)
            
            if sample['is_active'] == False:
                num_current_shutdown_switches += 1
            if could_shutdown:
                num_current_could_shutdown_switches += 1
    datacenters_switches.append(switches_per_datacenter)

In [45]:
# Powering down ports and switches connected to unused hosts and switches
num_new_shutdown_ports = 0
num_new_shutdown_switches = 0
datacenters_switches_copy = datacenters_switches.copy()
for index_datacenter, datacenter_switches in enumerate(datacenters_switches_copy):
    levels = 0
    for index_switch, switch in enumerate(datacenter_switches):
        if switch['role'] == "core switch":
            levels = switch['level']
    
    for level in range(levels):
        for index_switch, switch in enumerate(datacenter_switches):
            # First checking the corresponding effect of the resource aware placement of workloads
            # and for the capacity aware placement of data on the access switches
            if switch['role'] == "access switch":
                shutdown_switch = True
                for index_port, port in enumerate(switch['ports']):
                    if index_port != 0:
                        if port['host']['used_bw'] == 0:
                            port['is_active'] = False
                            num_new_shutdown_ports += 1
                        else:
                            port['is_active'] = True
                            shutdown_switch = False
                switch['is_active'] = shutdown_switch
                if shutdown_switch:
                    num_new_shutdown_switches += 1
            
            # Secondly checking the corresponding effect of the resource aware placement of workloads
            # and for the capacity aware placement of data on the distribution switches
            elif switch['role'] == "distribution switch" and switch['level'] == level:
                shutdown_switch = True
                for index_port, port in enumerate(switch['ports']):
                    if index_port != 0:
                        
                        if port['switch']['is_active'] == False:
                            port['is_active'] = False
                            num_new_shutdown_ports += 1
                        else:
                            port['is_active'] = True
                            shutdown_switch = False
                switch['is_active'] = shutdown_switch
                if shutdown_switch:
                    num_new_shutdown_switches += 1
            # Thirdly checking the corresponding effect of the resource aware placement of workloads
            # and for the capacity aware placement of data on the distribution switches
            elif switch['role'] == "core switch":
                shutdown_switch = True
                for index_port, port in enumerate(switch['ports']):
                    if port['switch']['is_active'] == False:
                        port['is_active'] = False
                        num_new_shutdown_ports += 1
                    else:
                        port['is_active'] = True
                        shutdown_switch = False # No network is being used
                switch['is_active'] = shutdown_switch
                if shutdown_switch:
                    num_new_shutdown_switches += 1

print("Number of port shutdowns with the previous placements:", num_current_shutdown_ports)
print("Number of possible ports to shutdown with the previous placements:", num_current_could_shutdown_ports)
print("Number of port shutdowns with the new placemenets:", num_new_shutdown_ports)

print("Number of switch shutdowns with the previous placements:", num_current_shutdown_switches)
print("Number of possible switch to shutdown with the previous placements:", num_current_could_shutdown_switches)
print("Number of switch shutdowns with the new placemenets:", num_new_shutdown_switches)

Number of port shutdowns with the previous placements: 0
Number of possible ports to shutdown with the previous placements: 187
Number of port shutdowns with the new placemenets: 394
Number of switch shutdowns with the previous placements: 0
Number of possible switch to shutdown with the previous placements: 0
Number of switch shutdowns with the new placemenets: 125


### Re-organizing Placement

### Optimal Placement

### Combining methods